In [1]:
import pandas as pd
data = pd.read_excel("../../../db/clean_orj_ulakbim.xlsx")
data.head()

,text,label
0,türk çimento sektörü ekonomik sosyal çevresel ...,Muhendislik
1,kollu galeri açma makineleri madencilik tünelc...,Muhendislik
2,özet araştırma derlenen bilgiler i̇stanbul ili...,Muhendislik
3,özet araştırma derlenen bilgiler i̇stanbul ili...,Muhendislik
4,özet çalışmada karadeniz yakalanan istavrit tr...,Muhendislik


In [2]:
data = data.dropna()

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2999 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2999 non-null   object
 1   label   2999 non-null   object
dtypes: object(2)
memory usage: 70.3+ KB


In [4]:
import gensim
review_text = data.text.apply(gensim.utils.simple_preprocess)
review_text

0       [türk, çimento, sektörü, ekonomik, sosyal, çev...
1       [kollu, galeri, açma, makineleri, madencilik, ...
2       [özet, araştırma, derlenen, bilgiler, stanbul,...
3       [özet, araştırma, derlenen, bilgiler, stanbul,...
4       [özet, çalışmada, karadeniz, yakalanan, istavr...
                              ...                        
2995    [çalışma, heracleum, platytaenium, boiss, öğre...
2996    [korunan, alanlar, ilgili, çalışmalar, eski, z...
2997    [çalışmada, bölgeye, uyumlarını, belirlemek, a...
2998    [çalışma, balıkesir, yöresindeki, sanayi, doma...
2999    [vertisol, latince, vertere, dönen, anlamına, ...
Name: text, Length: 2999, dtype: object

In [5]:
from gensim.models.doc2vec import TaggedDocument
class TaggedDocumentIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=doc, tags=[self.labels_list[idx]])
 
docLabels = data.label.tolist()
data_text = review_text.tolist()
sentences = TaggedDocumentIterator(data_text, docLabels)

In [6]:
model = gensim.models.Doc2Vec(
    vector_size=100,
    window=10,
    min_count=2,
    alpha=0.025,
    workers=11,
    epochs=20
)

In [7]:
model.build_vocab(sentences)

In [8]:
model.epochs

20

In [9]:
model.train(sentences,total_examples=model.corpus_count,epochs=model.epochs)

In [10]:
model.save("d2v_orj_ulakbim.doc2vec")

In [11]:
from gensim.models.doc2vec import Doc2Vec

model = Doc2Vec.load("d2v_orj_ulakbim.doc2vec")

In [12]:
model.wv.most_similar("zehir")

[('eph', 0.718512773513794),
 ('madenciliğinin', 0.704422116279602),
 ('spotu', 0.699924886226654),
 ('toza', 0.6994417905807495),
 ('planlaması', 0.6869729161262512),
 ('reklamlarına', 0.6810364127159119),
 ('pdr', 0.6701413989067078),
 ('kuaför', 0.6682091951370239),
 ('pratikte', 0.666111171245575),
 ('ekipman', 0.6478506326675415)]

In [14]:
model.wv.similarity("zehir","zehir")

1.0

In [16]:
model.wv.similarity("gönül","telefon")

0.38342604

In [17]:
model.infer_vector(["gönül"])

array([-0.03392114, -0.04867672, -0.07622264,  0.0308567 ,  0.09731833,
        0.01408746, -0.01457026,  0.00125277, -0.08328401,  0.00193161,
       -0.00857848, -0.01746358,  0.01325699,  0.0341536 , -0.0329453 ,
       -0.05340494,  0.02885413,  0.05066433, -0.04657393, -0.04169217,
       -0.01032519, -0.01842422, -0.04139296,  0.04124348,  0.0441225 ,
       -0.03873883, -0.01848763, -0.01761894, -0.05190233, -0.06283748,
        0.08318613,  0.05192014, -0.04219906, -0.06126979, -0.00494398,
        0.01669588,  0.00675383, -0.0223345 , -0.04753949,  0.00279279,
       -0.00896048, -0.06956822,  0.08899609, -0.04952392,  0.01743537,
       -0.06222329, -0.0187044 ,  0.00255656,  0.08203619, -0.03656359,
       -0.0238639 , -0.05641127, -0.02520956, -0.00187833, -0.02130456,
        0.04538967, -0.00884389,  0.0091948 , -0.04243005,  0.02952106,
        0.06197884,  0.02390523, -0.0004974 ,  0.00321191, -0.02629171,
       -0.00377455,  0.0074384 ,  0.01782338,  0.03978576, -0.03

In [18]:
model.wv["gönül"]

array([-0.8727408 , -0.1589908 , -1.7297758 , -0.0171827 ,  0.9744545 ,
        0.63576126, -0.7656876 ,  0.8791854 , -0.5302789 ,  0.28441942,
        0.17224874, -0.7388176 , -0.21828653,  0.5253572 , -0.4407429 ,
       -1.2519221 ,  0.7733206 ,  0.09351321, -0.18402825, -0.28168583,
       -0.4738092 ,  0.35108757, -0.21545728,  0.73857987, -0.08999285,
       -0.32088706, -0.35073674,  0.23765583, -1.474746  , -1.1967248 ,
        1.2594118 ,  0.40723634, -0.79347533, -0.41490182, -0.11232659,
       -0.08447611, -0.39305663, -0.73268044, -1.0863309 , -0.09197615,
       -0.894359  , -0.84434754,  0.86918855, -0.3515604 , -0.6625224 ,
       -0.662465  , -0.9401901 ,  0.01447485,  1.2326118 , -0.26450992,
       -0.59946764, -1.3435583 ,  0.21212247, -0.07113255, -0.37863603,
        0.7121684 ,  0.10248205,  0.8094365 , -0.5547925 , -0.1857814 ,
        0.8744469 ,  0.18858764, -0.27857774, -0.11192942, -0.45974636,
        0.6483926 , -0.24281467,  0.24539657,  0.78668535, -0.45